In [59]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import librosa 

In [60]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')

In [61]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

X_train_subset = X_train[:10000]
y_train_subset = y_train[:10000]

#### Converting the y values to Integer values rather than vectors 

In [62]:
y_train_final = []
for i in range(y_train_subset.shape[0]):
    if y_train_subset[i][0] == 1 :
        y_train_final.append(0)
    elif y_train_subset[i][1] == 1 :
        y_train_final.append(1)
    else:
        y_train_final.append(2)

y_train_final = np.array(y_train_final)
X_train_final = X_train_subset

In [63]:
print(X_train_final.shape, y_train_final.shape)

(10000, 513) (10000,)


### Creating the Model

In [64]:
# y = Wx + b
# 3x1 = (3,513) * (513,1) + (3,1)
W = np.random.rand(3, 513)
b = np.random.rand(3, 1)

In [65]:
def forward_pass(x, W, b):
    y_pred = np.array([np.zeros([3]) for i in range(x.shape[0])])

    for i in range(x.shape[0]): 
        y_pred[i] = (W.dot(x[i].reshape(-1,1)) + b).reshape(-1)

    y_pred_prob = np.array([np.zeros([3]) for i in range(y_pred.shape[0])])

    from scipy.special import softmax
    
    for i in range(y_pred.shape[0]):
        y_pred_prob[i] = softmax(y_pred[i])

    predictions = np.array([np.argmax(i) for i in y_pred_prob])

    return y_pred_prob, predictions

In [66]:
y_pred_prob, predictions = forward_pass(X_train_final, W, b)
print(predictions.shape)

(10000,)


In [67]:
def accuracy(predictions, target):
    correct = 0
    for i in range(len(predictions)):
        if predictions[i] == target[i]:
            correct += 1
    accuracy = correct/len(predictions)*100
    return accuracy

In [68]:
accuracy = accuracy(predictions, y_train_final) #calculating accuracy for our model
print(accuracy)

33.52


### Making the training and test sets 

In [69]:
# train test split 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_final, y_train_final, test_size=0.2, random_state=42)

### Training and Evaluating

In [70]:
def train(x,y,W,b,lr,epochs):
    for i in range(epochs):
        y_pred_prob, _ = forward_pass(x, W, b)
        y_pred_prob[np.arange(x.shape[0]),y] -= 1

        grad_W = y_pred_prob.T.dot(x) 
        grad_b = np.sum(y_pred_prob, axis = 0).reshape(-1,1)

        W -= (lr * grad_W)
        b -= (lr * grad_b)

    return W, b


In [71]:
W, b = train(X_train_final, y_train_final, W, b, 0.01, 300)


In [76]:
testProbabilities, testPredictions = forward_pass(X_test, W, b)

correctPreds = 0
for i in range(len(testPredictions)):
    if testPredictions[i] == y_test[i]:
        correctPreds += 1
acc = correctPreds / len(testPredictions) * 100
print("Model accuracy on test dataset - {}".format(acc))

Model accuracy on test dataset - 88.35


### Saving the Model

In [53]:
np.save('saved_model/mlr/W', W)
np.save('saved_model/mlr/b',b)